In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
# inten = pd.read_csv('data/depart_train.csv')
# sns.countplot(x="label", data=inten)

In [3]:
# inten = pd.read_csv('data/inten_train.csv')
# sns.countplot(x="label", data=inten)

In [43]:
import os
import torch
import torch.nn as nn

from utils import *
from transformers import BertTokenizer, BertForSequenceClassification
from torchtext.data import Field, LabelField, TabularDataset, BucketIterator, Iterator


class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()
        
        options_name = "bert-base-chinese"
        #BertForSequenceClassification.config_class = 14
        self.encoder = BertForSequenceClassification.from_pretrained(options_name,num_labels = 14)
    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]
        #print(self.encoder(text, labels=label))
        return loss, text_fea
    
model_inten = BERT().to(device)
load_checkpoint('inten_best_model.pt', model_inten)

class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()
        
        options_name = "bert-base-chinese"
        #BertForSequenceClassification.config_class = 14
        self.encoder = BertForSequenceClassification.from_pretrained(options_name,num_labels = 15)
    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]
        #print(self.encoder(text, labels=label))
        return loss, text_fea

model_depart = BERT().to(device)
load_checkpoint('depart_best_model.pt', model_depart)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Model loaded from <== inten_best_model.pt


Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Model loaded from <== depart_best_model.pt


0.0221283920109272

In [44]:
with open('inten_dic.json') as json_file:
    dic = json.load(json_file)
inten_dic = {v:k for k,v in zip(dic.keys(),dic.values())}

with open('depart_dic.json') as json_file:
    dic = json.load(json_file)
depart_dic = {v:k for k,v in zip(dic.keys(),dic.values())}

os.environ["CUDA_VISIBLE_DEVICES"]="0"
torch.cuda.get_device_name(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

label_field = Field(sequential=False, tokenize=y_tokenize, use_vocab=False, batch_first=True)
text_field  = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field),('titletext', text_field)]

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [45]:
def QA(Qu):
    df = pd.DataFrame()
    df['label'] = [0]
    df['titletext'] = [Qu]
    df.to_csv('data/inten_test.csv', index=False)
    # TabularDataset

    _,test = TabularDataset.splits(path='./', train='data/inten_valid.csv',test='data/inten_test.csv', format='CSV', fields=fields, skip_header=True)

    test_iter = BucketIterator(test, batch_size=64, sort_key=lambda x: len(x.titletext),
                                device=device, train=True, sort=True, sort_within_batch=True)

    test_loader = test_iter
    model_inten.eval()
    model_depart.eval()
    pred_inten,pred_depart = [],[]
    with torch.no_grad():
        for i in test_iter:
                labels = i.label
                titletext = i.titletext
                labels = labels.type(torch.LongTensor)           
                labels = labels.to(device)
                titletext = titletext.type(torch.LongTensor)  
                titletext = titletext.to(device)
                _, output_inten = model_inten(titletext, labels)
                _, output_depart = model_depart(titletext, labels)

                pred_inten.extend(torch.argmax(output_inten, 1).tolist())
                pred_depart.extend(torch.argmax(output_depart, 1).tolist())
            
    return inten_dic[pred_inten[0]],depart_dic[pred_depart[0]]

In [47]:
QA('企管系可以提前畢業嗎')

('畢業', '企業管理學系')

In [48]:
QA('產經系面試要戴口罩嗎')

('面試', '產業經濟學系')

In [49]:
QA('公行系有那些老師')

('學系介紹', '公共行政學系')

In [51]:
QA('統計系跟其他科系有什麼不同')

('學系介紹', '統計學系')